In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

print(sys.path)

import numpy as np
import pandas as pd
import cvxpy as cp

from src.setup_problem import Setup
from src.format_winston_data import WinstonDataFormatter_v2
from src.format_winston_data import generate_and_save_unsupervised_data
from src.misc import *

['/home/onoue/ws/lukasiewicz_1/notebooks/winston_2', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.11/site-packages', '/home/onoue/ws/lukasiewicz_1']


# 0. データの準備

In [3]:
file_name_list = os.listdir("./winston_pca_2/")
file_path_list = [os.path.join("./winston_pca_2", file_name) for file_name in file_name_list]

dfs = [pd.read_csv(file, index_col=0) for file in file_path_list]
df_data = pd.concat(dfs, ignore_index=True)

df_data

,animal,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,cheetah,-20.060238,-11.480357,15.623350,-6.305073,-6.696852,4.830261,-5.379857,2.432294,9.030500,...,0.125542,-0.080214,0.926857,-0.314778,-0.374476,-0.507503,0.318324,-0.128268,-0.045080,-0.206593
1,cheetah,-48.204088,18.591528,4.103486,2.440145,-10.566893,-14.992575,-2.351199,9.304754,-7.887125,...,-1.832162,-0.673048,1.983817,2.164188,0.928383,1.989706,0.480698,-1.720412,0.499422,-4.106438
2,cheetah,-13.188533,-23.096405,2.572728,-3.706356,2.869635,-3.104971,3.141484,1.542608,5.318094,...,-0.348495,-0.930289,0.061565,0.067395,-0.050295,-0.080876,0.121771,-0.493991,-0.457423,-0.195045
3,cheetah,-20.250505,-9.332849,-3.731214,-2.956475,-4.907211,-2.487805,-10.028531,1.898730,1.671651,...,-0.171502,-0.323352,0.199332,0.127779,-0.849878,-0.229659,-0.313232,0.213185,-1.336118,-0.310712
4,cheetah,-20.059971,-1.582472,-7.138032,-11.185771,18.417116,-1.680940,-6.145799,-0.107130,3.890427,...,1.681730,0.600582,2.152980,-2.025689,-0.199217,-0.165662,-0.718164,2.490273,0.523311,1.913032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,ostrich,31.291656,6.482612,-11.086591,-10.524747,4.290951,6.700974,21.181147,-8.942196,5.719353,...,-0.332219,-0.871387,-0.577859,-0.421761,0.225885,-0.944522,0.130902,-0.473095,0.188853,-0.326705
6496,ostrich,-6.391303,-17.068252,-1.496960,-14.264796,3.159807,-2.084721,8.642888,-2.086789,-8.758540,...,-2.444848,-1.743830,-1.234302,-1.398103,-3.151357,0.797208,-0.660319,0.189967,0.322385,-0.457151
6497,ostrich,-33.814540,10.456383,15.762074,6.935928,-1.560411,-3.322494,-3.718956,10.833294,6.694091,...,0.151620,0.986486,-1.042699,0.025971,-0.971935,-0.679953,0.292943,0.210233,0.598297,-0.516183
6498,ostrich,34.769526,21.752479,-3.132908,5.004080,3.544195,14.021153,-4.892887,1.884487,3.299574,...,-0.115667,-0.087014,0.304643,0.442750,0.269512,-0.045713,0.555787,-0.068632,0.019545,-0.239788


In [4]:
df_labels = pd.read_csv('./labels.csv', index_col=0).fillna(-1)
display(df_labels.head())
print(df_labels.shape)

,albatross,cheetah,ostrich,penguin,zebra
hair(x),-1.0,1.0,-1.0,-1.0,1.0
mammal(x),-1.0,1.0,-1.0,-1.0,1.0
milk(x),-1.0,1.0,-1.0,-1.0,-1.0
feather(x),1.0,-1.0,1.0,1.0,-1.0
bird(x),1.0,-1.0,1.0,1.0,-1.0


(31, 5)


In [15]:
data_formatter = WinstonDataFormatter_v2(df_data, df_labels)
data_formatter.format_data()
save_dir_path = os.path.join(project_dir_path, "inputs/winston_pca_full")
data_formatter.save_as_csv(save_dir_path)
generate_and_save_unsupervised_data(save_dir_path, data_num=30, data_dim=200)

Done!


In [5]:
def random_sample_by_label(df, label_column='animal', sample_size=10, selected_labels=None):
    # 選択されたラベルのみを対象にする
    if selected_labels is not None:
        df = df[df[label_column].isin(selected_labels)]

    # サンプルサイズがNoneの場合、元のデータをそのまま返す
    if sample_size is None:
        return df.reset_index(drop=True)

    # 各ラベル毎に指定の個数をランダムに抜き出す
    sampled_dataframes = []
    unique_labels = df[label_column].unique()

    for label in unique_labels:
        label_data = df[df[label_column] == label]
        sample_data = label_data.sample(n=sample_size, random_state=42)
        sampled_dataframes.append(sample_data)

    # 抜き出したデータを結合
    result_df = pd.concat(sampled_dataframes, ignore_index=True)

    return result_df


original_df = df_data.copy()
selected_labels = ['cheetah', 'albatross']  
sampled_df = random_sample_by_label(original_df, sample_size=100, selected_labels=selected_labels)

sampled_df

,animal,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,cheetah,15.402662,-3.807138,3.632292,32.764571,-5.664949,2.876672,3.040756,-8.643766,7.808112,...,-4.031143,0.502019,0.630617,0.088934,-0.005006,2.169094,-1.383973,-0.150240,-2.225031,-0.375203
1,cheetah,-50.593476,-7.922209,-11.033482,6.115892,2.230875,-2.827979,-8.589891,3.639380,-0.459745,...,0.152746,0.734783,0.300104,-0.220096,-0.999787,-0.102783,-0.280356,-0.086377,-0.098555,-0.002831
2,cheetah,-10.398780,49.083416,12.765650,3.406599,2.639825,-1.695449,-15.687951,-9.690763,-6.096559,...,-0.634467,-0.049114,-0.563111,0.451740,0.678812,0.386950,-0.803237,-0.082105,-0.050870,-0.782950
3,cheetah,-0.934629,-2.057008,-0.887415,-7.865467,-7.703866,-2.916751,1.805643,-4.821153,0.240899,...,-0.383386,0.378471,1.924444,1.428540,0.363502,0.191559,-1.472842,-2.003535,0.022197,-0.438671
4,cheetah,10.387551,2.727248,0.374913,3.122029,-1.818549,-7.941755,-2.613647,-8.021543,-0.653920,...,-1.009989,-1.500397,-0.375630,-1.125995,-0.887603,1.666718,-0.122017,-0.161629,0.449841,0.364122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,albatross,-10.554250,-10.180245,15.334719,1.620547,-4.822957,5.442124,2.169065,3.001244,-3.199404,...,0.026839,1.849687,-2.736205,0.890098,1.161233,-0.217144,-0.313529,-0.044662,0.352890,-0.387748
196,albatross,2.097137,31.900102,-2.181612,9.192555,0.074148,17.854490,-5.033813,-3.029893,-1.265763,...,0.172376,0.070216,-0.786730,-0.045865,0.129781,0.010598,-0.116206,0.678112,-0.207397,0.046922
197,albatross,-34.750480,5.682939,-2.522935,1.560119,4.492572,5.512418,-0.092059,13.299925,-8.100403,...,1.065749,2.863806,-0.142327,1.671186,-1.470173,-0.542646,0.696255,-0.044227,0.497618,-1.096086
198,albatross,-51.186763,-6.981031,-5.796971,-21.319474,7.340640,-0.598602,4.284274,-10.050741,-5.227979,...,1.667506,-0.502423,0.967173,1.614759,-0.238520,-0.303419,-0.560537,-0.460491,0.500064,-0.487871


In [6]:
data_formatter = WinstonDataFormatter_v2(sampled_df, df_labels)
data_formatter.format_data()
save_dir_path = os.path.join(project_dir_path, "inputs/winston_pca_100_two_animals")
data_formatter.save_as_csv(save_dir_path)
generate_and_save_unsupervised_data(save_dir_path, data_num=30, data_dim=200)

Done!


In [8]:
sampled_df.to_csv(os.path.join(save_dir_path, "base.csv"))

# 1. 

---

- 動物が cheetah と albatross のみ
- 各 predicate に対してデータ数は 100 個

In [19]:
data_dir_path = os.path.join(project_dir_path, 'inputs/winston_pca_100_two_animals')

file_list = os.listdir(data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_2']
}

problem_instance = Setup(data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.7598166465759277 seconds! 

Loading rules ...
Done in 0.0003383159637451172 seconds! 

Identifying predicates ...
Done in 0.001447439193725586 seconds! 

Constructing objective function ...
Done in 2.0502946376800537 seconds! 

Constructing constraints ...
Done in 6.776906490325928 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.11/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


(CVXPY) Nov 12 02:50:08 PM: Your problem has 12463 variables, 22380 constraints, and 0 parameters.
(CVXPY) Nov 12 02:50:10 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 12 02:50:10 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 12 02:50:10 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 12 02:50:13 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 12 02:50:13 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Nov 12 02:50:13 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 12 02:50:20 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 12 02:50:24 PM: A

In [20]:
def test_trained_predicate(predicates_dict, test_data_dict):
    result_dict = {}
    p_names = predicates_dict.keys()

    for p_name in p_names:
        pred_vals = []
        preds = []

        p = predicates_dict[p_name]
        test_data = test_data_dict[p_name]

        if isinstance(test_data, pd.DataFrame):
            test_data = np.array(test_data)

        cnt = 0

        for data in test_data:

            x, ans = data[:-1], data[-1]
            pred_val = p(x).value
            pred_vals.append(pred_val)

            if (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1):
                cnt += 1

            pred = (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1)
            preds.append(pred)

        p_arr = np.hstack([test_data, 
                           np.array(pred_vals).reshape(-1,1), 
                           np.array(preds).reshape(-1, 1)])
        
        # p_df = pd.DataFrame(p_arr, columns=['r', 'g', 'b', 'Ans', 'pred_val', 'pred'])

        result_dict[p_name] = p_arr

        print(cnt)
        print(f'Accuracy of {p_name}: {cnt / len(test_data)}')

    return result_dict

In [21]:
p_dict = problem_instance.predicates_dict
test_data = problem_instance.L

In [22]:
# 上記の辞書から特定のキーを抽出する
selected_predicates = ['cheetah(x)', 'albatross(x)']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}
selected_test_data = {key: value for key, value in test_data.items() if key in selected_predicates}

In [23]:
print(selected_p_dict)
print(selected_test_data)

{'cheetah(x)': <src.misc.Predicate object at 0x7fa95758c290>, 'albatross(x)': <src.misc.Predicate object at 0x7fa9575916d0>}
{'albatross(x)': array([[ 1.54026623e+01, -3.80713784e+00,  3.63229166e+00, ...,
        -2.22503050e+00, -3.75202968e-01, -1.00000000e+00],
       [-5.05934763e+01, -7.92220926e+00, -1.10334821e+01, ...,
        -9.85549264e-02, -2.83093228e-03, -1.00000000e+00],
       [-1.03987805e+01,  4.90834165e+01,  1.27656496e+01, ...,
        -5.08696178e-02, -7.82950033e-01, -1.00000000e+00],
       ...,
       [-3.47504800e+01,  5.68293865e+00, -2.52293507e+00, ...,
         4.97618060e-01, -1.09608599e+00,  1.00000000e+00],
       [-5.11867634e+01, -6.98103142e+00, -5.79697076e+00, ...,
         5.00063709e-01, -4.87871061e-01,  1.00000000e+00],
       [ 6.65699094e+00, -2.61752635e+01, -1.30839177e+01, ...,
        -7.36808937e-01, -1.00200723e-01,  1.00000000e+00]]), 'cheetah(x)': array([[ 1.54026623e+01, -3.80713784e+00,  3.63229166e+00, ...,
        -2.22503050e+0

In [24]:
res_dict = test_trained_predicate(selected_p_dict, selected_test_data)

193
Accuracy of cheetah(x): 0.965
193
Accuracy of albatross(x): 0.965


In [25]:
pd.DataFrame(res_dict['albatross(x)'])

,0,1,2,3,4,5,6,7,8,9,...,193,194,195,196,197,198,199,200,201,202
0,15.402662,-3.807138,3.632292,32.764571,-5.664949,2.876672,3.040756,-8.643766,7.808112,-9.994593,...,0.088934,-0.005006,2.169094,-1.383973,-0.150240,-2.225031,-0.375203,-1.0,-1.409972e-11,1.0
1,-50.593476,-7.922209,-11.033482,6.115892,2.230875,-2.827979,-8.589891,3.639380,-0.459745,-0.165427,...,-0.220096,-0.999787,-0.102783,-0.280356,-0.086377,-0.098555,-0.002831,-1.0,-4.668599e-11,1.0
2,-10.398780,49.083416,12.765650,3.406599,2.639825,-1.695449,-15.687951,-9.690763,-6.096559,4.259269,...,0.451740,0.678812,0.386950,-0.803237,-0.082105,-0.050870,-0.782950,-1.0,2.785089e-01,1.0
3,-0.934629,-2.057008,-0.887415,-7.865467,-7.703866,-2.916751,1.805643,-4.821153,0.240899,6.864183,...,1.428540,0.363502,0.191559,-1.472842,-2.003535,0.022197,-0.438671,-1.0,1.642910e-01,1.0
4,10.387551,2.727248,0.374913,3.122029,-1.818549,-7.941755,-2.613647,-8.021543,-0.653920,2.966750,...,-1.125995,-0.887603,1.666718,-0.122017,-0.161629,0.449841,0.364122,-1.0,-2.659084e-11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-10.554250,-10.180245,15.334719,1.620547,-4.822957,5.442124,2.169065,3.001244,-3.199404,-0.408772,...,0.890098,1.161233,-0.217144,-0.313529,-0.044662,0.352890,-0.387748,1.0,1.000000e+00,1.0
196,2.097137,31.900102,-2.181612,9.192555,0.074148,17.854490,-5.033813,-3.029893,-1.265763,-5.948295,...,-0.045865,0.129781,0.010598,-0.116206,0.678112,-0.207397,0.046922,1.0,1.000000e+00,1.0
197,-34.750480,5.682939,-2.522935,1.560119,4.492572,5.512418,-0.092059,13.299925,-8.100403,-6.282212,...,1.671186,-1.470173,-0.542646,0.696255,-0.044227,0.497618,-1.096086,1.0,1.000000e+00,1.0
198,-51.186763,-6.981031,-5.796971,-21.319474,7.340640,-0.598602,4.284274,-10.050741,-5.227979,-9.905559,...,1.614759,-0.238520,-0.303419,-0.560537,-0.460491,0.500064,-0.487871,1.0,1.000000e+00,1.0


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

data = pd.read_csv(os.path.join(data_dir_path, "base.csv"))

# 特徴量とラベルを分ける
X = data.drop('animal', axis=1)
y = data['animal']

# Random Forestモデルの作成と学習
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimatorsは適宜変更可能
rf_model.fit(X, y)

y_pred = rf_model.predict(X)

accuracy = accuracy_score(y, y_pred)
classification_rep = classification_report(y, y_pred)

In [18]:
print(accuracy)
print()
print(classification_rep)

1.0

              precision    recall  f1-score   support

   albatross       1.00      1.00      1.00       100
     cheetah       1.00      1.00      1.00       100

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [26]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

data = pd.read_csv(os.path.join(data_dir_path, "base.csv"))

# 特徴量とラベルを分ける
X = data.drop('animal', axis=1)
y = data['animal']

# SVMモデルの作成と学習
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)

y_pred = svm_model.predict(X)

accuracy = accuracy_score(y, y_pred)
classification_rep = classification_report(y, y_pred)

In [27]:
print(accuracy)
print()
print(classification_rep)

1.0

              precision    recall  f1-score   support

   albatross       1.00      1.00      1.00       100
     cheetah       1.00      1.00      1.00       100

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

